IMPORTS

In [22]:
from shapely.geometry import Point
import ee
import geopandas as gpd
import hashlib
import os
import pandas as pd
import random
import rasterio
import shutil
import urllib.request

CONFIG

In [23]:
# === CONFIG === ## Paths & File Extensions
CSV_PATH = "pasadena.csv"
BASE_DIR = "chips"
IMAGE_EXT = ".tif"  # or ".png" or ".tiff" or other common image file extension

# Directory Builder Labels - for neatness - go crazy with it
L1 = "256"
L2 = "512"
L3 = "30cm"
L4 = "60cm"
L5 = "RGB"
L6 = "RGBN"
L7 = "GT"
L8 = "2K"

# Image Directory Names - chip children
IMAGE_DIR_1 = f"pasadena_{L1}_{L4}_{L5}_{L7}{L8}"
IMAGE_DIR_2 = f"pasadena_{L1}_{L4}_{L5}_{L7}"
IMAGE_DIR_3 = f"final"

# Output Directory
OUTPUT_DIR = os.path.join(BASE_DIR, IMAGE_DIR_3)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# For Testing
TEST_STOP = 5  # this is a loop stop condition used for testing throughput

DATA

In [24]:
# === LOAD TREE POINT === #
df = pd.read_csv(CSV_PATH)
df.head(3)

,OBJECTID,Tree_Rec,Alt_Tree_ID,Common_Name,Genus,Species,Status_Text,House_Number,Street_Direction,Street_Name,Street_Type,Street_Suffix,Trunk_Diameter,X_Coordinate,Y_Coordinate,Classification_Text,x,y
0,2,69861,13,CHINESE ELM,ULMUS,PARVIFOLIA,Active,850.0,NaN,ADELAIDE,DR,NaN,16.0,6.528584e+06,1.880864e+06,Street Tree,6.528584e+06,1.880864e+06
1,3,69862,15,SAWTOOTH ZELKOVA,ZELKOVA,SERRATA,Active,858.0,NaN,ADELAIDE,DR,NaN,16.0,6.528584e+06,1.880894e+06,Street Tree,6.528584e+06,1.880894e+06
2,4,69863,16,CHINESE ELM,ULMUS,PARVIFOLIA,Active,859.0,NaN,ADELAIDE,DR,NaN,14.0,6.528544e+06,1.880891e+06,Street Tree,6.528544e+06,1.880891e+06


XY TO LAT LONG FUNCTION - Detailed info on worldwide coordinate systems can be found here: https://epsg.io/

In [25]:
def convert_coordinates(x, y, from_crs="EPSG:2229", to_crs="EPSG:4326"):
    """Converts a single X, Y projected coordinate pair to (lat, lon)."""
    point = gpd.GeoSeries([Point(x, y)], crs=from_crs).to_crs(to_crs)
    return point.y.iloc[0], point.x.iloc[0]  # returns (lat, lon) tuple

In [26]:
# Convert from X/Y to Lat/Long
df.columns = df.columns.str.lower() # all column names to lower
df[["lat", "lon"]] = df.apply(lambda row: pd.Series(convert_coordinates(row["x_coordinate"], row["y_coordinate"])), axis=1)
# df.head(1)

# Create a new geodetic pair
df["geodetic_pair"] = list(zip(df["lat"], df["lon"]))
df.head(1)

,objectid,tree_rec,alt_tree_id,common_name,genus,species,status_text,house_number,street_direction,street_name,...,street_suffix,trunk_diameter,x_coordinate,y_coordinate,classification_text,x,y,lat,lon,geodetic_pair
0,2,69861,13,CHINESE ELM,ULMUS,PARVIFOLIA,Active,850.0,NaN,ADELAIDE,...,NaN,16.0,6.528584e+06,1.880864e+06,Street Tree,6.528584e+06,1.880864e+06,34.160652,-118.109356,"(34.160651534763694, -118.10935561976741)"


In [27]:
coords = list(df["geodetic_pair"])

INITIALIZE AND TEST EARTH ENGINE

In [28]:
# === EE INIT === #
try:
    ee.Initialize(project="google_project_name-######")  # Your google project name goes here.
except Exception:
    ee.Authenticate()
    ee.Initialize()

THIS EE.IMAGECOLLECTION() SETUP GETS 60 CM AND WILL BE USED IN THE 60 CM VERSION OF get_chips() BELOW.

In [29]:
# === EE TEST 1=== #
random.seed(42)
sample_idx = random.randint(0,57069)
sample_idx = 0
coord = df.iloc[sample_idx]["geodetic_pair"]
point = ee.Geometry.Point(list(reversed(coord)))  # <--Note: the reversed coords requirement

image = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(point) \
    .filterDate('2018-01-01', '2020-12-31') \
    .sort('system:time_start', False) \
    .first()

display(image.getInfo())

{'type': 'Image',
 'bands': [{'id': 'R',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [10390, 12330],
   'crs': 'EPSG:26911',
   'crs_transform': [0.6, 0, 396060, 0, -0.6, 3783720]},
  {'id': 'G',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [10390, 12330],
   'crs': 'EPSG:26911',
   'crs_transform': [0.6, 0, 396060, 0, -0.6, 3783720]},
  {'id': 'B',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [10390, 12330],
   'crs': 'EPSG:26911',
   'crs_transform': [0.6, 0, 396060, 0, -0.6, 3783720]},
  {'id': 'N',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [10390, 12330],
   'crs': 'EPSG:26911',
   'crs_transform': [0.6, 0, 396060, 0, -0.6, 3783720]}],
 'version': 1747193923819894,
 'id': 'USDA/NAIP/DOQQ/m_3411856_sw_11_060_20200505',
 'pr

THIS EE.IMAGECOLLECTION() SETUP GETS 30 CM AND WILL BE USED IN THE 30 CM VERSION OF get_chips() BELOW.

In [30]:
# # === EE TEST 2 === #
# # Replace this with your actual point coordinates
# random.seed(42)
# sample_idx = random.randint(0,57069)
# sample_idx = 0
# coord = df.iloc[sample_idx]["geodetic_pair"]
# point = ee.Geometry.Point(list(reversed(coord)))  # <--Note: the reversed coords requirement
#
# # Load California boundary to constrain to state-level coverage (optional but useful)
# states = ee.FeatureCollection("TIGER/2018/States")
# california = states.filter(ee.Filter.eq("NAME", "California")).first()
#
# # Query NAIP imagery that intersects your point and is within California and the right date range
# image = ee.ImageCollection("USDA/NAIP/DOQQ") \
#     .filterBounds(california.geometry()) \
#     .filterDate("2021-01-01", "2024-12-31") \
#     .sort("system:time_start", False) \
#     .first()
#
# # Optionally check the GSD to confirm it's 0.3 (30 cm)
# gsd = image.select('R').projection().nominalScale()
# print("Nominal GSD (meters):", gsd.getInfo())
# print(image.getInfo())

MAIN CHIP GENERATION

60 cm GSD VERSION

In [31]:
# CHIP GENERATION CELL 1.60
def get_chip(coord, meters=153.6, pixel_dim=256):
    """
    Generate a NAIP image chip centered on `coord`, ~meters side, as `pixel_dim` × `pixel_dim`.
    coord: [lat, lon]
    meters: length of one side in meters (112m ≈ 0.07 mi)
    pixel_dim: size of the output square image
    Returns: URL to download/display PNG chip
    """
    center = ee.Geometry.Point(list(reversed(coord)))
    region = center.buffer(meters / 2).bounds()
    point = ee.Geometry.Point(list(reversed(coord)))  # <--Note: the reversed coords requirement

    ### FOR cm GSD
    '''
    .filterDate("2020-01-01", "2022-12-31") \
    '''
    image = ee.ImageCollection('USDA/NAIP/DOQQ') \
        .filterBounds(point) \
        .filterDate('2018-01-01', '2020-12-31') \
        .sort('system:time_start', False) \
        .first()


    return image.getThumbURL({
        'region': region,
        'dimensions': f'{pixel_dim}x{pixel_dim}',
        'format': 'GeoTIFF',
        'bands': ['R', 'G', 'B'],
        'min': 0,
        'max': 255,
    })

30 cm GSD VERSION

In [32]:
# # # CHIP GENERATION CELL 1.30
# def get_chip(coord, meters=128, pixel_dim=512): # meters at 128 instead of 153.6
#     center = ee.Geometry.Point(list(reversed(coord)))
#
#     # Load California boundary (needed to force 30cm tiles)
#     states = ee.FeatureCollection("TIGER/2018/States")
#     california = states.filter(ee.Filter.eq("NAME", "California")).first()
#
#     # Load NAIP image over that location and time
#     image = ee.ImageCollection("USDA/NAIP/DOQQ") \
#         .filterBounds(center) \
#         .filterBounds(california.geometry()) \
#         .filterDate("2021-01-01", "2024-12-31") \
#         .sort("system:time_start", False) \
#         .first()
#
#     # gsd = image.select('R').projection().nominalScale()
#     # print("Detected GSD (meters):", gsd.getInfo())
#     #
#     # if image is None:
#     #     print(f"⚠️ No image found at {coord}")
#     #     return None
#
#     # Build a safe region — intersected with image footprint to avoid blank tiles
#     base_region = center.buffer(meters / 2).bounds(1)
#     image_geom = image.geometry()
#     region = base_region.intersection(image_geom, ee.ErrorMargin(1))  # final guardrail
#
#     # Return PNG thumbnail URL
#     return image.clip(region).getThumbURL({
#         'region': region,
#         'dimensions': f'{pixel_dim}x{pixel_dim}',
#         'format': 'png',
#         'bands': ['R', 'G', 'B'],
#         'min': 0,
#         'max': 255,
#     })


In [33]:
# CHIP GENERATION CELL 2
'''
Use the coordinate list to generate chips for MVP demo
'''
# Save chips for MVP demo

for i, pair in enumerate(coords):
    try:
        url = get_chip(pair)
        filename = os.path.join(OUTPUT_DIR, f"image_{i:04d}{IMAGE_EXT}")
        urllib.request.urlretrieve(url, filename)
        # print(f"Saved {filename}")
    except Exception as e:
        print(f"Failed on index {i}: {e}")
    if i == TEST_STOP: break  # COMMENT THIS LINE OUT WHEN NOT TESTING THROUGH PUT
print(f"Saved {i + 1} images")

Saved 6 images


# UTILITY FUNCTIONS AND POST GEN DIAGNOSTICS

## CRASH RECOVERY - IF IMAGE GENERATION IS INTERRUPTED, ACTIVATE RECOVERY CELLS 1 & 2, FOLLOW INSTRUCTIONS IN RECOVERY CELL 2

In [34]:
## RECOVERY CELL 1 - FOLLOW "Image-gen crash recovery MADE EASY", CLEAR, RERUN
'''
# Image-gen crash recovery MADE EASY
# visually inspect image dir to get image_count after a break and/or inspect the filename of the last image saved and note the digits in the image file name.
# stopped_at = the digits in the image file name
'''
stopped_at_image = 5
continued_at = stopped_at_image + 1
leftover_coords = coords[continued_at:]
len(coords[continued_at:])

57064

In [35]:
## RECOVERY CELL 2 - COMMENT OUT CHIP GENERATION CELL 2, FOLLOW INSTRUCTIONS IN RECOVERY CELL 1
'''
Repeating the same loop, but with the left over slice
'''
i = 5
for i, pair in enumerate(leftover_coords):
    try:
        url = get_chip(pair)
        filename = os.path.join(OUTPUT_DIR, f"image_{i + continued_at:04d}{IMAGE_EXT}")

        urllib.request.urlretrieve(url, filename)
        # print(f"Saved {filename}")
    except Exception as e:
        print(f"Failed on index {i}: {e}")
    if i == TEST_STOP: break  # COMMENT THIS LINE OUT WHEN NOT TESTING THROUGH PUT
print(f"Saved {i + 1} images")

Saved 6 images


SPOT CHECK IMAGE FORMAT - FILENAMES ARE SEQUENTIAL

In [36]:
# TEST_IMAGE_1 = os.path.join(OUTPUT_DIR, f"image_{(random.randint(0, len(coords) - 1)):04d}{IMAGE_EXT}")   # IMAGE IDX IS USED IN THE FILENAME AND IS AN INT ON LINESPACE 0 - LEN(DATA) - 1
TEST_IMAGE_1 = os.path.join(OUTPUT_DIR, f"image_{(random.randint(0, TEST_STOP*2)):04d}{IMAGE_EXT}")
TEST_IMAGE_1

'chips\\final\\image_0001.tif'

In [37]:
def compare_metas(sample_geo, i, img_tup):
    with rasterio.open(sample_geo) as src:
        metadata = src.meta
        tags = src.tags()
        print(f"***********************************FOR TEST IMAGE {img_tup[i]}*******************************")
        print("Basic metadata:", metadata)
        print("Full tags (including CRS info):", tags)
        print("\n")

In [38]:
geo_samples = [TEST_IMAGE_1]
for i, sample in enumerate(geo_samples):compare_metas(sample, i, (1,3))

***********************************FOR TEST IMAGE 1*******************************
Basic metadata: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 256, 'height': 256, 'count': 3, 'crs': CRS.from_wkt('PROJCS["NAD83 / UTM zone 11N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26911"]]'), 'transform': Affine(0.6040460015702592, 0.0, 397666.9570184235,
       0.0, -0.6051055637064564, 3780611.3559153015)}
Full tags (including CRS info): {'TIFFTAG_XRESOL

GAP CHECK/FIXER - filenames are sequential - we use this fact to hunt for missing images

TO CHECK FOR/FIX GAPS IN FILE GEN - ACTIVATE GAP CELL 1 AND FOLLOW THE INSTRUCTIONS

In [39]:
## GAP CELL 1 - RUN THIS CELL (COLD OR HOT).
## IF THERE IS 1 OR MORE LINES OF --> "Gap between: 19810 and 19812 (missing index: 19811)" PRINTED, ACTIVATE GAP CELL 2 AND PROCEED
## NO PRINT == NO GAPS IN IMAGE GENERATION
# Load and sort file indices
image_dir = OUTPUT_DIR
file_list = sorted(
    [int(f.split("_")[1].split(".")[0]) for f in os.listdir(image_dir) if f.endswith(f"{IMAGE_EXT}")]
)
gap_list = []
# Scan for gap
for i in range(1, len(file_list)):
    if file_list[i] - file_list[i - 1] > 1:
        gap_list.append(int(file_list[i - 1] + 1))
        print(f"Gap between: {file_list[i - 1]} and {file_list[i]} (missing index: {file_list[i - 1] + 1})")

In [40]:
## GAP CELL 1 - ONLY RUN THIS CELL IF THERE IS 1 OR MORE LINES OF --> "Gap between: 19810 and 19812 (missing index: 19811)" PRINTED BY GAP CELL 1
## Note: FROM A COLD START (EMPTY NAMESPACE), DEACTIVATE "CHIP GENERATION CELL 2" AND RUN THE BOOK FROM THE TOP TO RECOVER MISSING CHIPS
for gap in gap_list:
    pair = coords[gap]
    url = get_chip(pair)
    filename = os.path.join(OUTPUT_DIR, f"image_{gap:04d}{IMAGE_EXT}")
    urllib.request.urlretrieve(url, filename)
    print(f"Recovered: image_{gap:04d}{IMAGE_EXT}")

CHECK FOR DUPLICATES IMAGES

In [41]:
# check for duplicate images
def file_md5(path):
    with open(path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

image_dir = OUTPUT_DIR
hash_to_original = {}
duplicate_map = {}

for fname in sorted(os.listdir(image_dir)):  # Sort for consistent "first is original"
    if fname.endswith(f"{IMAGE_EXT}"):
        path = os.path.join(image_dir, fname)
        h = file_md5(path)
        if h in hash_to_original:
            duplicate_map[fname] = hash_to_original[h]
        else:
            hash_to_original[h] = fname

print(f"Total duplicates: {len(duplicate_map)}")

# Preview or export
for dup, orig in duplicate_map.items():
    print(f"{dup} is a duplicate of {orig}")


Total duplicates: 0


MASS RENAME (IF NEEDED)

In [42]:
# # Paths
# source_dir = "path"  # Directory with the original .tiff files
# image_titles = df["image_title"].tolist()
#
# # List and sort files
# files = sorted([f for f in os.listdir(source_dir) if f.endswith('.tiff')])
#
# # Destination dir (you can use the same as source_dir if needed, or point to a new one)
# target_dir = "path"
# os.makedirs(target_dir, exist_ok=True)
#
# # Rename process
# count_by_name = {}  # Will track usage of a given new_filename
#
# for i, old_filename in enumerate(files):
#     base_name = image_titles[i]           # New filename candidate
#     candidate_name = base_name            # Will be modified if duplicated
#
#     if candidate_name in count_by_name:
#         # We've seen this name before, increment suffix
#         count_by_name[base_name] += 1
#         candidate_name = f"{base_name}_{count_by_name[base_name]}"
#     else:
#         # First time seeing this name
#         count_by_name[base_name] = 0
#
#     new_filename = candidate_name
#     old_path = os.path.join(source_dir, old_filename)
#     new_path = os.path.join(target_dir, new_filename)
#
#     shutil.copy2(old_path, new_path)
#
# # Done
# print(f"Renamed and copied {len(files)} files to {target_dir}")


# END OF FILE